In [ ]:
%store -r
%store

# Interactuando con campañas

In [27]:
import time
from time import sleep
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 50)         # Keep the output on one page
pd.set_option('display.max_colwidth', 200)  


from datetime import datetime
import uuid
import random

import boto3
import botocore
from botocore.exceptions import ClientError
import pandas as pd

In [28]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

In [29]:
anime = pd.read_csv(data_dir + '/anime.csv',index_col='MAL_ID')
anime.sample(1)

,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
MAL_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
30262,Okane ga Nai Specials,6.12,"Comedy, Romance, Yaoi",No Money Omake episodes,お金がないっ,Special,4,"Feb 9, 2007 to Dec 21, 2007",Unknown,"KlockWorx, Happinet Pictures",Media Blasters,Unknown,Manga,1 min. per ep.,R+ - Mild Nudity,Unknown,7143,3924,6,177,2632,61,112,942,143.0,94.0,150.0,264.0,315.0,221.0,103.0,51.0,44.0,79.0


# Funcion para verificar la existencia del anime

In [30]:
def get_anime_by_id(anime_id, df=anime):
    try:
        return df.loc[int(anime_id)]['Name']
    except:
        return "Error obtaining title"

def get_animerow_by_id(anime_id, df=anime):
    try:
        return df.loc[int(anime_id)]
    except:
        return "Error obtaining title"

def get_anime_like(name, df=anime):
    try:
        found = df[df.Name.apply(lambda x: name.lower() in x.lower())].sample().index.values[0]
        return found
    except:
        return "Error no hay"

def get_anime_equal(name, df=anime):
    try:
        found = df[df.Name.apply(lambda x: name.lower() == x.lower())].sample().index.values[0]
        return found
    except:
        return "Error no hay"

# Similar Items (SIMS)

In [31]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, anime_ID):
    # Get the movie name
    anime_name = get_anime_by_id(anime_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(anime_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        tanime = get_anime_by_id(item['itemId'])
        recommendation_list.append(tanime)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [anime_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [108]:
sims_recommendations_df = pd.DataFrame()
animes = [get_anime_equal('fullmetal alchemist'), get_anime_equal('death note'), get_anime_equal('Hellsing Ultimate'), get_anime_equal('One Punch Man')]

for a in animes:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, a)

sims_recommendations_df[:15]

,Fullmetal Alchemist,Death Note,Hellsing Ultimate,One Punch Man
0,HORIZON feat. Hatsune Miku,Fate/Zero,Hellsing: The Dawn,One Punch Man 2nd Season
1,Fullmetal Alchemist: The Conqueror of Shamballa,Hellsing Ultimate,Black Lagoon: Roberta's Blood Trail,Boku no Hero Academia
2,Septem Charm Magical Kanan,Overlord,Drifters,Boku no Hero Academia 3rd Season
3,Mahou Shoujo-tai Arusu the Adventure,Boku no Hero Academia,Fate/Zero,Overlord
4,Fullmetal Alchemist: Premium Collection,Sen to Chihiro no Kamikakushi,Redline,Mob Psycho 100 II
5,Ken En Ken: Aoki Kagayaki,Tokyo Ghoul:re,Durarara!!x2 Ten,Shokugeki no Souma: San no Sara
6,Gunslinger Girl: Il Teatrino OVA,NHK ni Youkoso!,Kara no Kyoukai 3: Tsuukaku Zanryuu,Re:Zero kara Hajimeru Isekai Seikatsu
7,Code Geass: Boukoku no Akito 3 - Kagayaku Mono Ten yori Otsu Picture Drama,One Punch Man 2nd Season,Sidonia no Kishi,Kono Subarashii Sekai ni Shukufuku wo! 2
8,Yuugo: Koushounin,Toki wo Kakeru Shoujo,Mushishi: Hihamukage,Drifters
9,Bleach: The Sealed Sword Frenzy,Kimetsu no Yaiba,Mob Psycho 100 II,Tate no Yuusha no Nariagari


# User Personalization


In [33]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        anime = get_anime_by_id(item['itemId'])
        recommendation_list.append(anime)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [109]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in [1,293,29738, 3990, 283878]:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users[:10]

,1,293,29738,3990,283878
0,Mob Psycho 100 II,Aldnoah.Zero 2nd Season,Naruto,Aldnoah.Zero 2nd Season,Steins;Gate Movie: Fuka Ryouiki no Déjà vu
1,Haikyuu!!: Karasuno Koukou vs. Shiratorizawa Gakuen Koukou,Fate/stay night: Unlimited Blade Works 2nd Season - Sunny Day,Dragon Ball Z,Death Note,Death Note
2,5-toubun no Hanayome,Gosick,Fullmetal Alchemist,Drifters,Shingeki no Kyojin
3,Haikyuu!!: To the Top 2nd Season,Carnival Phantasm: HibiChika Special,Cowboy Bebop,Durarara!!x2 Ten,Sword Art Online
4,Naruto: Shippuuden Movie 6 - Road to Ninja,Kokoro Connect: Michi Random,Clannad: After Story,Sengoku Basara Ni,One Punch Man
5,Steins;Gate: Oukoubakko no Poriomania,Darling in the FranXX,Code Geass: Hangyaku no Lelouch R2,Sidonia no Kishi,Shigatsu wa Kimi no Uso
6,Dororo,Working!!!: Lord of the Takanashi,Angel Beats!,Hellsing Ultimate,Boku no Hero Academia
7,Shingeki no Kyojin Season 3 Part 2,Saenai Heroine no Sodatekata: Ai to Seishun no Service-kai,Fruits Basket,Fate/stay night: Unlimited Blade Works 2nd Season,Noragami
8,Naruto: Shippuuden Movie 5 - Blood Prison,Musaigen no Phantom World,Bishoujo Senshi Sailor Moon,Ichiban Ushiro no Daimaou,Fate/stay night: Unlimited Blade Works
9,Hotarubi no Mori e,Isshuukan Friends.,Rurouni Kenshin: Meiji Kenkaku Romantan,Arslan Senki (TV): Fuujin Ranbu,Log Horizon


# Usando Filtros

In [35]:
def get_new_recommendations_df_by_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        anime = get_anime_by_id(item['itemId'])
        recommendation_list.append(anime)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

## Veamos las recomendaciones por todas los géneros que cuentan con filtros..

In [110]:
usuario  = 30000

recommendations_df_shelves = pd.DataFrame()
for filter_arn in meta_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, usuario, filter_arn)

recommendations_df_shelves[:10]

,Action,Adventure,Comedy,Drama,Fantasy,Kids,Music,Sci-Fi,Shounen
0,Fate/Zero,Toki wo Kakeru Shoujo,Yuru Yuri Nachuyachumi!+,Maria-sama ga Miteru Specials,Fate/Zero,Pokemon Movie 01: Mewtwo no Gyakushuu,Love Live! School Idol Project 2nd Season,Strike Witches OVA,Fairy Tail OVA
1,Akuma no Riddle: Shousha wa Dare? Nukiuchi Test,Little Witch Academia,Fairy Tail OVA,Maria-sama ga Miteru: Haru Specials,Fairy Tail OVA,Pokemon: The Origin,K-On!!,Toki wo Kakeru Shoujo,Shingeki no Kyojin OVA
2,Black★Rock Shooter (OVA),Murder Princess,Akuma no Riddle: Shousha wa Dare? Nukiuchi Test,Mahou Shoujo Madoka★Magica Movie 2: Eien no Monogatari,"Koihime†Musou: Gunyuu, Seitoukaichou no Za wo Neratte Aiarasou no Koto - Ato, Porori mo Aru yo!",Pokemon Movie 02: Maboroshi no Pokemon Lugia Bakutan,Love Live! School Idol Project OVA,Mahou Shoujo Lyrical Nanoha: The Movie 1st,Haikyuu!! Second Season
3,Strike Witches OVA,Fairy Tail x Rave,Yuru Yuri Nachuyachumi!,Black★Rock Shooter (OVA),Valkyrie Drive: Mermaid,Smile Precure! Movie: Ehon no Naka wa Minna Chiguhagu!,K-On!: Live House!,Suzumiya Haruhi no Shoushitsu,Gabriel DropOut
4,Akuma no Riddle,Flip Flappers,Non Non Biyori Repeat,Gosick,Shingeki no Kyojin OVA,Pokemon Movie 03: Kesshoutou no Teiou Entei,K-On!!: Keikaku!,Strike Witches: Operation Victory Arrow,Boku no Hero Academia
5,Kämpfer für die Liebe,Sen to Chihiro no Kamikakushi,"Yuru Yuri: Doushite, Tomaranai, Tokimeki, Dokidoki, Paradox, Eternal",Maria-sama ga Miteru 3rd,Little Witch Academia,Digimon Adventure: Bokura no War Game!,Bokura no Live Kimi to no Life,Strike Witches Movie,Gintama Movie 1: Shinyaku Benizakura-hen
6,Mahou Shoujo Lyrical Nanoha: The Movie 1st,Little Witch Academia: Mahoujikake no Parade,Kämpfer für die Liebe,Kuttsukiboshi,Murder Princess,Precure All Stars Movie New Stage 2: Kokoro no Tomodachi,Zombieland Saga,Mahou Shoujo Lyrical Nanoha: The Movie 2nd A's,Shinryaku! Ika Musume
7,Valkyrie Drive: Mermaid,Shin Koihime†Musou: Otome Tairan,Shoujo Sect,Kimi no Na wa.,Ichiban Ushiro no Daimaou,Dokidoki! Precure Movie: Mana Kekkon!!? Mirai ni Tsunagu Kibou no Dress,Gravitation,Darling in the FranXX,Fairy Tail x Rave
8,Shingeki no Kyojin OVA,Koihime†Musou,Maria-sama ga Miteru 3rd Specials,Sekaiichi Hatsukoi 2,Mai-Otome Specials,Heartcatch Precure! Movie: Hana no Miyako de Fashion Show... Desu ka!?,Love Live! The School Idol Movie,Gintama Movie 1: Shinyaku Benizakura-hen,Shingeki no Kyojin
9,Strike Witches: Operation Victory Arrow,Kono Subarashii Sekai ni Shukufuku wo!,K-On!!,Toki wo Kakeru Shoujo,Hina Logi: From Luck & Logic,Futari wa Precure: Splash☆Star Movie - Tick Tack Kiki Ippatsu!,K-On! Movie,Date A Live,To LOVE-Ru OVA


El siguiente tema son los eventos en tiempo real. Personalizar tiene la capacidad de escuchar eventos de su aplicación para actualizar las recomendaciones que se muestran al usuario. Esto es especialmente útil en cargas de trabajo de medios, como videos, donde la intención de un cliente puede diferir según si lo está viendo con sus hijos o por su cuenta.

Además, los eventos que se registran a través de este sistema se almacenan hasta que se emite una llamada de eliminar, y se utilizan como datos históricos junto con los demás datos de interacción que proporcionó cuando entrena sus próximos modelos.

# Eventos en tiempo real (Event tracker)

empezamos creando un evento

In [92]:
response = personalize.create_event_tracker(
    name='AnimeTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

bcb3c3ea-8ebe-48be-98fa-abeed0b43322


In [52]:
session_dict = {}

def rate_anime(USER_ID, ITEM_ID,EVENT_VALUE, EVENT_TYPE='RATING' ):
    """
    Simula una calificación
    para enviarla a Amazon personalize como un evento. 
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Hace la llamada
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'eventValue': int(EVENT_VALUE),
        'properties': event_json
        }]
    )

In [53]:
def get_new_recommendations_df_users_real_time(user_id):
    # Get the artist name (header of column)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        anime = get_anime_by_id(item['itemId'])
        recommendation_list.append(anime)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = ['Recomendaciones'])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    return new_rec_DF

In [54]:
rating_complete = pd.read_csv(data_dir + '/rating_complete.csv', usecols=['user_id'])

In [55]:
rating_complete['user_id'].unique().max()

353404

# Cualquier usuario sobre 353404 es nuevo ;)

In [115]:
#Cliente abre su cuenta de video streaming.
user_id = 2000000
get_new_recommendations_df_users_real_time(user_id)[:10]

,Recomendaciones
0,Naruto
1,Dragon Ball Z
2,Fullmetal Alchemist
3,Cowboy Bebop
4,Clannad: After Story
5,Code Geass: Hangyaku no Lelouch R2
6,Angel Beats!
7,Fruits Basket
8,Bishoujo Senshi Sailor Moon
9,Rurouni Kenshin: Meiji Kenkaku Romantan


In [116]:
# Cliente ve y califica Hunter X Hunter
anime_name = 'Fullmetal Alchemist'
nota = 9
anime_id = get_anime_equal(anime_name)
print (f'Calificando {anime_name}({anime_id}) con nota {nota}')
rate_anime(user_id, anime_id, nota)

Calificando Fullmetal Alchemist(121) con nota 9


In [117]:
get_new_recommendations_df_users_real_time(user_id)[:10]

,Recomendaciones
0,Naruto
1,Cowboy Bebop
2,Code Geass: Hangyaku no Lelouch R2
3,Fruits Basket
4,Dragon Ball Z
5,Fullmetal Alchemist: Brotherhood
6,Clannad: After Story
7,Elfen Lied
8,Angel Beats!
9,InuYasha


In [118]:
# Cliente ve y califica Steins;Gate
anime_name = 'Elfen Lied'
nota = 7
anime_id = get_anime_equal(anime_name)
print (f'Calificando {anime_name}({anime_id}) con nota {nota}')
rate_anime(user_id, anime_id, nota)

Calificando Elfen Lied(226) con nota 7


In [119]:
get_new_recommendations_df_users_real_time(user_id)[:10]

,Recomendaciones
0,Fullmetal Alchemist: Brotherhood
1,Naruto
2,Code Geass: Hangyaku no Lelouch R2
3,Mirai Nikki
4,Angel Beats!
5,School Days
6,Clannad: After Story
7,Durarara!!
8,Bakemonogatari
9,Cowboy Bebop


In [120]:
# Cliente ve y califica ranma
anime_name = 'Naruto'
nota = 6
anime_id = get_anime_like(anime_name)
print (f'Calificando {anime_name}({anime_id}) con nota {nota}')
rate_anime(user_id, anime_id, nota)

Calificando Naruto(34566) con nota 6


In [121]:
get_new_recommendations_df_users_real_time(user_id)[:10]

,Recomendaciones
0,Fullmetal Alchemist: Brotherhood
1,Mirai Nikki
2,Sword Art Online
3,Death Note
4,Shingeki no Kyojin
5,Angel Beats!
6,Tokyo Ghoul
7,One Punch Man
8,Hunter x Hunter (2011)
9,Neon Genesis Evangelion


In [122]:
# Cliente ve y califica shingeki no kyojin
anime_name = 'Hunter x Hunter (2011)'
nota = 8
anime_id = get_anime_like(anime_name)
print (f'Calificando {anime_name}({anime_id}) con nota {nota}')
rate_anime(user_id, anime_id, nota)

Calificando Hunter x Hunter (2011)(11061) con nota 8


In [123]:
get_new_recommendations_df_users_real_time(user_id)[:10]

,Recomendaciones
0,Fullmetal Alchemist: Brotherhood
1,Shingeki no Kyojin
2,One Punch Man
3,Sword Art Online
4,Death Note
5,Mirai Nikki
6,Tokyo Ghoul
7,Shingeki no Kyojin Season 2
8,Ao no Exorcist
9,Another


## Rest API para POC Amazon Personalize

El objetivo de este proyecto es un despliegue rápido de una API RESTfull para invocar las campañas de Amazon Personalize. Esto lo haremos con Cloud Development Kit (CDK) en Python.

Este proyecto está tal cual (as-is) para probar las predicciones de Amazon Personalize utilizando API Gateway y Lambda.

https://github.com/ensamblador/recomendaciones-rest-api